In [13]:
import sagemaker
from sagemaker.pytorch import PyTorch
import pickle
import os
import boto3
import sagemaker
import torch
import argparse
import pickle
# import mlflow
import numpy as np
import json

import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torchvision import models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# # SageMaker Session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# # Estimator
# estimator = PyTorch(
#     entry_point='evaluation_script.py',  # Değerlendirme scriptiniz
#     role=role,
#     framework_version='2.0.0',
#     py_version='py310',
#     instance_count=1,
#     instance_type='ml.g5.xlarge',
#     sagemaker_session=sagemaker_session
# )

# # Değerlendirme işini başlatma
# estimator.fit({'test': 's3://your-bucket/test-data/'})

# # Değerlendirme raporunu S3'ten alın
# evaluation_report_s3_uri = estimator.model_data


In [ ]:
best_model_path = 's3://your-bucket/models/model_resnet18.pth'  # Adjust this path based on the extracted files
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load(best_model_path, map_location=device))

In [ ]:
test_metadata_path = 's3://wildfires/data/test/datajob/test.pkl'
test_metadata = pickle.dump(test_metadata_path)

In [ ]:
class FireDataset(Dataset):
    def __init__(self, metadata, train_dir="", transform=None):
        self.metadata = metadata
        self.transform = transform
        self.train_dir = train_dir

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        # img_path = self.metadata[idx]['local_path']
        img_path = os.path.basename(self.metadata[idx]['image_location'])
        img_path = os.path.join(self.train_dir, img_path)
        image = Image.open(img_path).convert('RGB')
        label = self.metadata[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
val_test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

test_dataset = FireDataset(test_metadata, train_dir=args.train_dir, transform=val_test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)


In [ ]:
def test(model, test_loader, loss_function, epoch, device, run):
    model = model.to(device)
    loss_function = loss_function.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [1]:
%pwd

'/home/sagemaker-user/Steps/Job 4 evulation'

In [17]:
import boto3

s3 = boto3.client('s3')

s3.download_file('wildfires', 'sagemaker/fire-image-classification/test.pkl', 'test.pkl')


with open('test.pkl', 'rb') as file:
    test_metadata = pickle.load(file)

# test_metadata

In [13]:
import boto3


s3 = boto3.client('s3')

s3.download_file('wildfires', 'models/model_resnet18.tar.gz', 'model_resnet18.tar.gz')



In [14]:
import tarfile

tar_file_path = 'model_resnet18.tar.gz'  # Açmak istediğiniz tar arşiv dosyasının yolu

# Tar dosyasını açma
try:
    with tarfile.open(tar_file_path, 'r:gz') as tar:
        tar.extractall(path='extracted_folder/')  # Arşivi çıkartılacak klasör yolunu belirtin
    print(f'Tar file "{tar_file_path}" successfully extracted.')
except Exception as e:
    print(f'Error extracting tar file "{tar_file_path}": {e}')


Tar file "model_resnet18.tar.gz" successfully extracted.


In [23]:
print(tar_file_path.split('.')[0])

model_resnet18


In [ ]:
's3://wildfires/sagemaker/fire-image-classification/fire.506.png'

In [ ]:
%pwd

In [25]:
s3.upload_file(
    f'images.jpeg',
    BUCKET_NAME,
    f"sagemaker/fire-image-classification/fire.506.png",
)

In [48]:
s3 = boto3.client('s3')

BUCKET_NAME = 'wildfires'

s3.upload_file(
    f'evaluation.py',
    BUCKET_NAME,
    f"scripts/evaluation.py",
)

In [49]:
import sagemaker
import boto3
from sagemaker.processing import ScriptProcessor
from sagemaker import get_execution_role
from sagemaker import image_uris

role = get_execution_role()
sm_session = sagemaker.Session()

boto_session = boto3.Session()
region = boto_session.region_name

constants = {
    "bucket_name": "wildfires",
    # "sklearn_image_uri_version": "1.2-1",
    "region": boto_session.region_name
}

script_path = sm_session.upload_data(
    path='evaluation.py',
    bucket=constants["bucket_name"],
    key_prefix="scripts"
)

test_metadata_path = 's3://wildfires/sagemaker/fire-image-classification'

best_model_path = 's3://fire-project-hs/34/36ba5f52b9944395ab2ccf94aed56e5a/artifacts/models'

model_package_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'

# sklearn_image_uri = image_uris.retrieve(
#         framework="sklearn",
#         region=constants["region"],
#         version=constants["sklearn_image_uri_version"],
#     )

script_processor = ScriptProcessor(
    role=role,
    image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
    command=['python3'],
    instance_count=1,
    instance_type='ml.m5.large'
)

script_processor.run(
    job_name='wildfire-evaluation-job-t21',
    code=script_path,
    arguments=[
        '--test_metadata_bucket', constants["bucket_name"],
        '--best_model_bucket', constants["bucket_name"],
        '--test_metadata_prefix', 'sagemaker/fire-image-classification',
        '--best_model_prefix', 'models',
        '--test_metadata_file', 'test.pkl',
        '--best_model_file', 'model_resnet18.tar.gz',
        '--result_bucket', constants["bucket_name"],
        '--result_prefix', 'evaluation/dummy/',
        '--data_dir', 's3://wildfires/sagemaker/fire-image-classification',
        '--model_package_arn', model_package_arn,
        '--region', constants["region"]
    ]
)

INFO:sagemaker:Creating processing-job with name wildfire-evaluation-job-t21


................................Collecting sagemaker
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 87.0 MB/s eta 0:00:00
[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To updat

UnexpectedStatusException: Error for Processing job wildfire-evaluation-job-t21: Failed. Reason: AlgorithmError: , exit code: 1

In [ ]:
# import sagemaker
# from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

# sagemaker_session = sagemaker.Session()
# role = sagemaker.get_execution_role()

# script_processor = ScriptProcessor(
#     image_uri='763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.7.1-cpu-py36-ubuntu18.04',  # Kullanacağınız image
#     role=role,
#     instance_count=1,
#     instance_type='ml.m5.xlarge'
# )

# script_processor.run(
#     code='PATH-FOR-EVALUATION.PY',
#     inputs=[
#         ProcessingInput(
#             source='PATH-FOR-TEST-DATASET and METADATA',
#             destination='/opt/ml/processing/input'
#         ),
#         ProcessingInput(
#             source='PATH-FOR-MODEL',
#             destination='/opt/ml/processing/model'
#         )
#     ],
#     outputs=[
#         ProcessingOutput(
#             source='/opt/ml/processing/output',
#             destination='PATH-FOR-EVALUATION RESULTS'
#         )
#     ]
# )


In [ ]:
import boto3


sm = boto3.client('sagemaker')

model_package_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'

response = s3_client.get_object(Bucket=constants["bucket_name"], Key='evaluation/dummy/')

evaluation_result = response['Body'].read().decode('utf-8') 


sm.update_model_package(
    ModelPackageArn=model_package_arn,
    MetadataProperties={
        'EvaluationResults': json.dumps(evaluation_result)
    }
)


In [5]:
import boto3


sm = boto3.client('sagemaker')

In [4]:
boto_session.region_name

'eu-central-1'

In [3]:
boto_session = boto3.Session()


In [6]:
sm

In [34]:
result = {"metrics": {"accuracy": 0.0, "test_loss": 0.0}}

In [42]:
list(result["metrics"]['accuracy'])

TypeError: 'float' object is not iterable

In [75]:
result['metrics']

{'accuracy': 0.0, 'test_loss': 0.0}

In [54]:
tuple(json.dumps(result).split(" "))

('{"metrics":', '{"accuracy":', '0.0,', '"test_loss":', '0.0}}')

In [50]:
s3.download_file('wildfires', 'evaluation/dummy/', '48')

In [87]:
model_metrics = {
    'ModelQuality': {
        'Statistics': {
            'ContentType': 'application/json',
            'Statistics': json.dumps(result)
        }
    }
}

In [92]:
sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus='Approved',  # veya ihtiyaç duyulan onay durumu
    ApprovalDescription='Model evaluation results added',
    AdditionalInferenceSpecificationsToAdd={
        'ModelMetrics': [
            model_metrics
        ]
    }
)

ParamValidationError: Parameter validation failed:
Invalid type for parameter AdditionalInferenceSpecificationsToAdd, value: {'ModelMetrics': [{'ModelQuality': {'Statistics': {'ContentType': 'application/json', 'Statistics': '{"metrics": {"accuracy": 0.0, "test_loss": 0.0}}'}}}]}, type: <class 'dict'>, valid types: <class 'list'>, <class 'tuple'>